In [1]:
import numpy as np
import math
import re
from collections import Counter
from functools import cmp_to_key

In [2]:
with open("11-input", "r") as file:
    lines = file.readlines()
data_raw = [line.replace("\n", "") for line in lines]
data_raw = "\n".join(data_raw)
data_raw

'...#...........#..............#..................................................#...................#...................#..................\n................................................................................................#..............#...........................#\n..................................#...................#.......................................................................#.............\n.......................#....................................................................#...............................................\n........#........................................................#.......#..................................................................\n..............................................#............#......................................................#.........................\n...............#.....................#...............................#...........................#..........................#...............\n....#

In [36]:
test_data_raw = """...#......
.......#..
#.........
..........
......#...
.#........
.........#
..........
.......#..
#...#....."""

def preprocess_data (data):
    rows = [list(row) for row in data.split("\n")]
    return np.array(rows, dtype='U10')

test_data = preprocess_data(test_data_raw)
test_data

array([['.', '.', '.', '#', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '#', '.', '.'],
       ['#', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '#', '.', '.', '.'],
       ['.', '#', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '#'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '#', '.', '.'],
       ['#', '.', '.', '.', '#', '.', '.', '.', '.', '.']], dtype='<U10')

In [37]:
data = preprocess_data(data_raw)
data

array([['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '#'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ...,
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '#', ..., '.', '.', '.']], dtype='<U10')

In [7]:
universes_x, universes_y = np.where(test_data == '#')
universes_x, universes_y = list(universes_x), list(universes_y) 


([0, 1, 2, 4, 5, 6, 8, 9, 9], [3, 7, 0, 6, 1, 9, 7, 0, 4])

In [30]:

def solution (data):
    # expand universe
    universes_x, universes_y = np.where(data == '#')
    universes_x, universes_y = list(universes_x), list(universes_y) 

    expand_x = [i for i in range(data.shape[0]) if i not in universes_x]
    expand_y = [i for i in range(data.shape[1]) if i not in universes_y]

    start = 0
    slices = []
    for row in expand_x:
        slice = data[start:row,:]
        slices.append(slice)
        slices.append(data[row:row+1,:])
        start = row

    # add end of map
    slice = data[start:,:]
    slices.append(slice)

    expanded_universe = np.concatenate(slices, axis=0)

    start = 0
    slices = []
    for col in expand_y:
        slice = expanded_universe[:,start:col]
        slices.append(slice)
        slices.append(expanded_universe[:,col:col+1])
        start = col

    # add end of map
    slice = expanded_universe[:,start:]
    slices.append(slice)

    expanded_universe = np.concatenate(slices, axis=1)
    # print(expanded_universe)

    # shortest paths
    galaxies = np.where(expanded_universe == '#')
    galaxies = list(zip(galaxies[0], galaxies[1]))

    for idx, galaxy in enumerate(galaxies):
        expanded_universe[galaxy] = str(idx)
    # print(expanded_universe)

    num_galaxies = len(galaxies)
    tmp = np.zeros((num_galaxies, num_galaxies))
    indices = np.tril_indices_from(tmp, k=-1)
    galaxy_pairs = list(zip(indices[1], indices[0]))

    shortest_paths = []
    for galaxy_pair in galaxy_pairs:
        galaxy_a = galaxies[galaxy_pair[0]]
        galaxy_b = galaxies[galaxy_pair[1]]
        path = np.abs(galaxy_a[0] - galaxy_b[0]) + np.abs(galaxy_a[1] - galaxy_b[1])
        shortest_paths.append(path)

    # print(galaxy_pairs)
    # print(shortest_paths)

    return shortest_paths


sol = solution(data)
# sol = solution(test_data)
# display(sol)

display(sum(sol))

# print(np.ceil(solution.astype(int).max()/2))

9947476

In [44]:

def solution (data):
    # expand universe
    universes_x, universes_y = np.where(data == '#')
    universes_x, universes_y = list(universes_x), list(universes_y) 

    expand_x = [i for i in range(data.shape[0]) if i not in universes_x]
    expand_y = [i for i in range(data.shape[1]) if i not in universes_y]
    
    scaling_factor = 1000000

    # print(expanded_universe)

    # shortest paths
    galaxies = np.where(data == '#')
    galaxies = list(zip(galaxies[0], galaxies[1]))

    # for idx, galaxy in enumerate(galaxies):
    #     data[galaxy] = str(idx)
    # print(expanded_universe)

    num_galaxies = len(galaxies)
    tmp = np.zeros((num_galaxies, num_galaxies))
    indices = np.tril_indices_from(tmp, k=-1)
    galaxy_pairs = list(zip(indices[1], indices[0]))

    # print(data)
    shortest_paths = []
    for galaxy_pair in galaxy_pairs:
        galaxy_a = galaxies[galaxy_pair[0]]
        galaxy_b = galaxies[galaxy_pair[1]]
        # print("Galaxy a", galaxy_a)
        # print("Galaxy b", galaxy_b)

        rows_crossed = list(range(min(galaxy_a[0], galaxy_b[0]), max(galaxy_a[0], galaxy_b[0])))
        multiplier_rows = {row:1 for row in rows_crossed}
        rows_multiplied = list(set(rows_crossed).intersection(set(expand_x)))
        for row in rows_multiplied:
            multiplier_rows[row] = scaling_factor

        cols_crossed = list(range(min(galaxy_a[1], galaxy_b[1]), max(galaxy_a[1], galaxy_b[1])))
        multiplier_cols = {col:1 for col in cols_crossed}
        rows_multiplied = list(set(cols_crossed).intersection(set(expand_y)))
        for row in rows_multiplied:
            multiplier_cols[row] = scaling_factor

        path = sum(list(multiplier_rows.values())) + sum(list(multiplier_cols.values()))
        shortest_paths.append(path)

    # print(galaxy_pairs)
    print(shortest_paths)

    return shortest_paths


sol = solution(data)
# sol = solution(test_data)
# display(sol)

display(sum(sol))

# print(np.ceil(solution.astype(int).max()/2))

[12, 27, 15, 5000073, 5000061, 5000046, 6000092, 6000080, 6000065, 1000019, 7000111, 7000099, 7000084, 2000038, 1000019, 5000089, 5000077, 5000062, 16, 1000005, 2000024, 6000103, 6000091, 6000076, 1000030, 11, 1000010, 1000014, 8000129, 8000117, 8000102, 3000056, 2000037, 1000018, 3000040, 2000026, 33, 21, 6, 5000044, 6000063, 7000082, 5000058, 6000072, 8000098, 1000052, 1000040, 1000025, 4000025, 5000044, 6000063, 4000039, 5000053, 7000079, 1000019, 8000117, 8000105, 8000090, 3000044, 2000025, 1000006, 3000028, 2000014, 14, 8000084, 7000065, 23, 11, 10, 5000056, 6000075, 7000094, 5000070, 6000084, 8000110, 12, 1000031, 8000096, 5000087, 5000075, 5000060, 14, 1000011, 2000030, 6, 1000020, 3000046, 5000054, 4000035, 3000032, 5000064, 9, 11, 26, 5000072, 6000091, 7000110, 5000086, 6000100, 8000126, 28, 1000047, 8000112, 16, 5000080, 2000064, 2000052, 2000037, 3000017, 4000036, 5000055, 3000031, 4000045, 6000071, 2000031, 1000012, 6000057, 2000041, 3000025, 2000055, 4000070, 4000058, 4000

519939907614